Importing modules.

In [47]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import pickle

Defining a function that gives the approximate distance for a lattitude and longitude difference. This function does **not** take into account local curvature.

In [48]:
def approx_dist(lat: float, lon: float) -> float:
    dx = lat * 110.574 # km
    dy = lon * 111.320 * np.cos(lat) # km
    return np.sqrt(dx**2 + dy**2)

Imorting data and adding informations for later.

In [49]:
from get_recensement import get_recensement

recensement = get_recensement(relative_path="../../")

recensement

# # Loading data
# recensement = pd.read_excel(io="../../1832_v4.xlsx", na_values=["·","?"])
# classes = pd.read_excel(io="../../histoire urbaine digitale classification metiers.xlsx", sheet_name="data",  na_values="·")
# rues = pd.read_csv("../../nom_rues_et_coor.csv", sep=",")

# # Adding the 'chef_vocation_categorie' column
# classes = classes.rename(columns={"JOB": "chef_vocation_norm_2"})
# classes = classes.rename(columns={"CLASS": "chef_vocation_categorie"})
# recensement = recensement.merge(classes, on="chef_vocation_norm_2", how="outer")

# # Split the values on the '/' character
# split_vals = recensement['chef_vocation_categorie'].str.split('/')

# # Create a new dataframe from the split values
# recensement = recensement.assign(chef_vocation_categorie=split_vals).explode('chef_vocation_categorie')

# # Adding coordinates
# correspondances = {
#     "nom_rue": [
#         "ale", 
#         "etraz", 
#         "rue du pre", 
#         "chaucrau", 
#         "st laurent", 
#         "st pierre", 
#         "st etienne", 
#         "bourg", 
#         "st francois", 
#         "georgette",
#     ],
#     "nom_rue_norm_2": [
#         "rue de l'ale",
#         "rue d etraz",
#         "rue du pre",
#         "rue de chaucrau",
#         "place de st laurent",
#         "rue de st pierre",
#         "st etienne",
#         "rue de bourg",
#         "place de st froncois",
#         "chemin de georgette",
#     ]
# }
# for index, row in tqdm(recensement.iterrows(), total=len(recensement), desc="Adding coordinates"):
#     nom_rue = row["nom_rue_norm_2"]
#     if type(nom_rue) == float: continue # if nom_rue is NaN
#     if nom_rue in correspondances["nom_rue"]:
#         index = correspondances["nom_rue"].index(nom_rue)
#         nom_rue = correspondances["nom_rue_norm_2"][index]
#         corresponding_streets = rues.query("Nom_rue == @nom_rue")
#     else:
#         corresponding_streets = rues.query("Nom_rue.str.contains(@nom_rue)")
#     occurences = len(corresponding_streets)
#     if occurences == 1:
#         recensement.at[index, "rue_x"] = corresponding_streets["X"].values[0]
#         recensement.at[index, "rue_y"] = corresponding_streets["Y"].values[0]
#         continue
# print(f'{len(recensement.query("~rue_x.isnull()")) / len(recensement) * 100:.2f}% des personnes positionnées')

,Unnamed: 0,nom_rue,nom_rue_htr_corr,nom_rue_norm,nom_rue_norm_2,no_maison,proprietaire_nom,proprietaire_nom_corr,proprietaire_nom_htr_corr,proprietaire_nom_norm,...,pensionnaires_condition_htr_corr,pensionnaires_condition_norm,pensionnaires_recepisse,observations,Page,Division,chef_vocation_categorie,origine_region,rue_lon,rue_lat
0,1.0,martheray,martheray,marterey,marterey,1,veidmann,veidmann,veidmann,weidmann,...,petite fille naturelle|dom|macon|ouvrier charron,petite fille naturelle|domestique|maracon|ouvr...,12879|10913|16222,de barhara taidman nee,1.0,1-2,NaN,Vaud,6.628506,46.523442
1,309.0,villamond,villamond,villamond,villamont,NaN,NaN,NaN,NaN,NaN,...,savz intendane,savz intendane,NaN,NaN,12.0,1-2,artisanat,Vaud,6.638049,46.519400
2,1504.0,de marelice,de marelice,de marelice,moulin de pepinet,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,63.0,7-8,NaN,Vaud,6.635005,46.520948
3,3050.0,barre,barre,barre,barre,NaN,NaN,NaN,NaN,NaN,...,dque,domestique,NaN,NaN,117.0,16,NaN,Vaud,6.630860,46.522781
4,3299.0,ouchy,ouchy,ouchy,ouchy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,129.0,17,agricole,Vaud,6.630608,46.522297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3670,3133.0,lous le signas,lous le signas,lous le signas,barre,42,berthoud,berthoud,berthoud,berthoud,...,NaN,NaN,NaN,NaN,121.0,16,NaN,NaN,6.635930,46.525039
3671,3134.0,lous le signas,lous le signas,lous le signas,barre,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,121.0,16,NaN,NaN,6.635930,46.525039
3672,3182.0,ouchy,ouchy,ouchy,ouchy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,125.0,17,service,NaN,6.626871,46.507624
3673,3470.0,boissentil,boissentil,boissentil,bois-gentil,NaN,chevalley,chevalley,chevalley,chevalley,...,NaN,NaN,NaN,NaN,138.0,17,NaN,NaN,NaN,NaN


Filtering data. Mainly remove lines that have no vocation & no coordinates.

In [50]:
original_size = len(recensement)

recensement = recensement.query("not chef_vocation_categorie.isna()")
print(f"Size is now {len(recensement)/original_size*100:.0f}% of the original.")

#recensement = recensement.query("not rue_x.isna()")
#print(f"Size is now {len(recensement)/original_size*100:.0f}% of the original.")

recensement = recensement.query("not Division.isna()")
print(f"Size is now {len(recensement)/original_size*100:.0f}% of the original.")

recensement = recensement.query("not origine_region.isna()")
print(f"Size is now {len(recensement)/original_size*100:.0f}% of the original.")


Size is now 79% of the original.
Size is now 65% of the original.
Size is now 44% of the original.


A `networkx` graph is made.

In [51]:
G = nx.Graph()
for index, row in recensement.iterrows():
    G.add_node(
        index,
        nom_rue=row["nom_rue_norm_2"],
        proprietaire=row["proprietaire_nom_norm_2"],
        chef_nom=row["chef_nom_norm_2"],
        chef_annee_naissance=row["chef_annee_naissance"],
        chef_origine=row["chef_origine_norm_2"],
        chef_vocation=row["chef_vocation_norm_2"],
        chef_vocation_categorie=row["chef_vocation_categorie"],
        division=row["Division"],
        origine_region=row["origine_region"],
    )
    
number_of_unique_Divisions = len(recensement["Division"].value_counts())
number_of_unique_origine_regions = len(recensement["origine_region"].value_counts())
    
for index1, row1 in tqdm(recensement.iterrows(), total=len(recensement)):
    if pd.isna(row1["chef_vocation_categorie"]): continue
    for index2, row2 in recensement.iterrows():
        if pd.isna(row2["chef_vocation_categorie"]): continue
        if index2 < index1: continue

        score = 0

        #if row1["nom_rue_norm_2"] == row2["nom_rue_norm_2"]: score += 1
        #if row1["chef_origine_norm_2"] == row2["chef_origine_norm_2"]: score += 1

        # distance = approx_dist(lat=row1["rue_lat"] - row2["rue_lat"], lon=row1["rue_lon"] - row2["rue_lon"])
        # if distance > 0.01: continue
        
        if row1["Division"] == row2["Division"]:
            Division = row1["Division"]
            people_in_same_Division = len(recensement) / number_of_unique_Divisions
            score += 2 / (people_in_same_Division * (people_in_same_Division + 1))
        if row1["origine_region"] == row2["origine_region"]:
            origine_region = row1["origine_region"]
            peuple_with_same_origine_region = len(recensement) / number_of_unique_origine_regions
            score += 2 / (peuple_with_same_origine_region * (peuple_with_same_origine_region + 1))
        
        if score > 0: G.add_edge(index1, index2, weight=score)

100%|██████████| 1683/1683 [00:51<00:00, 32.96it/s]


Show some stats.

In [55]:
print(f"Number of nodes: {len(G.nodes)}")
print(f"Number of edges: {len(G.edges)}")

Number of nodes: 1575
Number of edges: 629741


Write `.gexf` file.

In [56]:
nx.write_gexf(G, 'individus_origine_rue.gexf')

In [57]:
len(recensement.query("Division == '1-2'"))

263